<a href="https://colab.research.google.com/github/Kaveesha20/MYSLT-app-Feature-Request/blob/developer/feature_requests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Scraping MYSLT App Reviews

In [1]:
!pip install google-play-scraper pandas  # Installs Python packages,scrape Google Play Store reviews,DataFrames
from google_play_scraper import reviews_all, Sort #fetch all available reviews,sort reviews
import pandas as pd

# Fetch MYSLT App Reviews
reviews = reviews_all(
    'com.slt.selfcare',  # MYSLT App ID
    lang='en',# Only English reviews
    country='lk', # Sri Lanka
    sort=Sort.NEWEST,# Get newest reviews first
    sleep_milliseconds=1000, # Delay to avoid being blocked
)
# Convert to DataFrame and rename 'at' to 'date'
df = pd.DataFrame(reviews)
if 'at' in df.columns:
    df.rename(columns={'at': 'date'}, inplace=True)
df['source'] = 'Google Play'

# Save to CSV
df = pd.DataFrame(reviews)
df.to_csv('myslt_reviews.csv', index=False)

# Download the CSV file
from google.colab import files
files.download('myslt_reviews.csv')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 407.7 kB/s eta 0:00:00


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [2]:

# Print the extracted reviews
for review in reviews:
    print(f"User: {review['userName']}")
    print(f"Rating: {review['score']}")
    print(f"Review: {review['content']}")
    print("-" * 20)

User: Vasanth Raj
Rating: 1
Review: disappointed to see that the app doesn't offer data usage tracking, forecasting bill payments option is secondary what's important is knowing how much data we have used.
--------------------
User: Shanith Kavinda
Rating: 1
Review: full of bugs. always something error massage comes up.
--------------------
User: Chamith Malshan
Rating: 1
Review: data remaining not show.
--------------------
User: Dushyanth Kandiah
Rating: 3
Review: everything ok, some features can be better like if you're going to upgrade a package if you click the upgrade button without any confirmation if upgrades the package
--------------------
User: Dilum Perera
Rating: 2
Review: The app technically works, I can pay my bills and see my usage stats but everything else is just below average. The design somehow looks like an early 2010s webpage, the colors are not eye pleasing, options are cluttered everywhere, no security at all, some of the options (changing mobile no, notificatio

In [4]:
!pip install app_store_scraper


In [5]:
import requests
import pandas as pd

def fetch_app_store_reviews(app_id="1492064957", pages=2):
    reviews = []
    for page in range(1, pages+1):
        url = f"https://itunes.apple.com/rss/customerreviews/page={page}/id={app_id}/sortby=mostrecent/json"
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to fetch page {page}")
            continue
        data = response.json()
        entries = data.get('feed', {}).get('entry', [])[1:]  # Skip app metadata
        for entry in entries:
            reviews.append({
                'userName': entry['author']['name']['label'],
                'content': entry['content']['label'],
                'score': entry['im:rating']['label'],
                'date': entry['updated']['label'],
                'source': 'App Store'
            })
    return reviews

# Fetch and save reviews
app_reviews = fetch_app_store_reviews()
df = pd.DataFrame(app_reviews)
df.to_csv('myslt_appstore_reviews.csv', index=False)

# Download the CSV file
files.download('myslt_appstore_reviews.csv')

# Print sample reviews
for review in app_reviews[:5]:
    print(f"User: {review['userName']}")
    print(f"Rating: {review['score']}")
    print(f"Review: {review['content']}")
    print("-" * 20)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

User: HemanthaJ
Rating: 1
Review: I don’t know why this developer doesn’t look other competitors application.Functionals are very poor and it makes frustrate the users.
--------------------
User: Mali(&(&(&(
Rating: 1
Review: Bill outstanding is not updating even though I paid the full bill amount
--------------------
User: Mali(&(&(&(
Rating: 1
Review: Total Payable is not even updating accurately eventhough i fully settled the monthly bill. mybill section in slt.lk is better than this app
--------------------
User: Susa.abey
Rating: 1
Review: A useless app. Never works.
--------------------
User: RareLime.exe
Rating: 1
Review: Yes. It’s not functioning properly
--------------------


# Detect and Translate Sinhala Reviews

In [6]:
!pip install langdetect deep-translator pandas

import pandas as pd
from langdetect import detect
from deep_translator import GoogleTranslator
import logging
from google.colab import files

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Initialize translator
translator = GoogleTranslator(source='auto', target='en')

# Translation function
def translate_review(text):
    try:
        if not text or pd.isna(text):
            return text
        if detect(text) != 'en':
            translated = translator.translate(text)
            logging.info(f"Translated: {text} -> {translated}")
            return translated
        return text
    except Exception as e:
        logging.warning(f"Translation failed for text: {text}. Error: {e}")
        return text

# Preprocess reviews
def preprocess_reviews(google_csv, appstore_csv):
    try:
        # Load Google Play reviews
        df_google = pd.read_csv(google_csv)
        if 'at' in df_google.columns:
            df_google.rename(columns={'at': 'date'}, inplace=True)
        df_google['source'] = 'Google Play'
        df_google = df_google[['userName', 'content', 'score', 'date', 'source']].dropna(subset=['content'])

        # Load App Store reviews
        df_appstore = pd.read_csv(appstore_csv)
        df_appstore = df_appstore[['userName', 'content', 'score', 'date', 'source']].dropna(subset=['content'])

        # Combine reviews
        df = pd.concat([df_google, df_appstore], ignore_index=True)
        logging.info(f"Loaded {len(df)} reviews from both stores")

        # Apply translation
        df['translated_content'] = df['content'].apply(translate_review)

        # Parse dates
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df['year'] = df['date'].dt.year

        # Save preprocessed reviews
        df.to_csv('myslt_preprocessed_reviews.csv', index=False)
        files.download('myslt_preprocessed_reviews.csv')
        logging.info("Saved preprocessed reviews to myslt_preprocessed_reviews.csv")

        return df

    except Exception as e:
        logging.error(f"Error preprocessing reviews: {e}")
        raise

# Example usage
if __name__ == "__main__":
    try:
        df = preprocess_reviews('myslt_reviews.csv', 'myslt_appstore_reviews.csv')
        print("Preprocessing complete. Downloaded: myslt_preprocessed_reviews.csv")
        print("\nSample of preprocessed reviews:")
        print(df[['userName', 'content', 'translated_content', 'source']].head())
    except Exception as e:
        print(f"Failed to preprocess reviews: {e}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.6 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=927f275b24e3b480c71b20a91682614c83ca06bfbb0d497bb1b20ea786499081
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Preprocessing complete. Downloaded: myslt_preprocessed_reviews.csv

Sample of preprocessed reviews:
            userName                                            content  \
0        Vasanth Raj  disappointed to see that the app doesn't offer...   
1    Shanith Kavinda  full of bugs. always something error massage c...   
2    Chamith Malshan                           data remaining not show.   
3  Dushyanth Kandiah  everything ok, some features can be better lik...   
4       Dilum Perera  The app technically works, I can pay my bills ...   

                                  translated_content       source  
0  disappointed to see that the app doesn't offer...  Google Play  
1  full of bugs. always something error massage c...  Google Play  
2                           data remaining not show.  Google Play  
3  everything ok, some features can be better lik...  Google Play  
4  The app technically works, I can pay my bills ...  Google Play  


In [7]:
!pip install transformers pandas

from transformers import pipeline
import pandas as pd
import logging
from google.colab import files

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load model
try:
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)
    logging.info("Model loaded successfully")
except Exception as e:
    logging.error(f"Failed to load model: {e}")
    raise

# Refined categories for MYSLT
main_categories = [
    "UI/UX Enhancement",
    "Payment Features",
    "Authentication",
    "Language Support",
    "Notifications",
    "Data Usage Tracking",
    "Customer Support",
    "Other"
]

category_mapping = {
    "UI/UX Enhancement": "UI/UX",
    "Payment Features": "Payment",
    "Authentication": "Login",
    "Language Support": "Language",
    "Notifications": "Notifications",
    "Data Usage Tracking": "Data Usage",
    "Customer Support": "Support",
    "Other": "Other"
}

sub_categories = {
    "UI/UX Enhancement": ["dark mode", "theme customization", "font size adjustment", "navigation improvements", "simplified layout"],
    "Payment Features": ["new payment methods", "autopay setup", "recurring payments", "bill reminders", "card management"],
    "Authentication": ["fingerprint login", "face recognition", "remember me", "multi-user login", "password recovery"],
    "Language Support": ["Sinhala support", "Tamil support", "multi-language support", "language selection"],
    "Notifications": ["mute notifications", "custom alert tones", "email notifications", "SMS alerts", "notification scheduling"],
    "Data Usage Tracking": ["data balance display", "usage history", "real-time tracking", "data plan management"],
    "Customer Support": ["live chat", "FAQ access", "ticket submission", "call support integration"],
    "Other": ["general feature request", "miscellaneous"]
}

# Helper functions
def is_feature_request(review, threshold=0.8):
    try:
        if not review or pd.isna(review):
            return False
        result = classifier(review, ["feature request", "bug report", "general feedback"])
        return result['labels'][0] == "feature request" and result['scores'][0] >= threshold
    except Exception as e:
        logging.warning(f"Feature request classification failed for review: {review}. Error: {e}")
        return False

def classify_feature_request(review, threshold=0.7):
    try:
        if not review or pd.isna(review):
            return {
                "review": review,
                "main_category": "Other",
                "sub_category": "miscellaneous",
                "main_confidence": 0.0,
                "sub_confidence": 0.0
            }
        main_result = classifier(review, main_categories)
        verbose_label = main_result['labels'][0]
        main_score = main_result['scores'][0]
        main_category = category_mapping.get(verbose_label, "Other")

        if main_category == "Other" and main_score < 0.7:
            main_result = classifier(review, main_categories[:-1])
            verbose_label = main_result['labels'][0]
            main_score = main_result['scores'][0]
            main_category = category_mapping.get(verbose_label, "Other")

        sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
        sub_result = classifier(review, sub_list)
        sub_category = sub_result['labels'][0]
        sub_score = sub_result['scores'][0]

        return {
            "review": review,
            "main_category": main_category,
            "sub_category": sub_category,
            "main_confidence": round(main_score, 2),
            "sub_confidence": round(sub_score, 2)
        }
    except Exception as e:
        logging.warning(f"Category classification failed for review: {review}. Error: {e}")
        return {
            "review": review,
            "main_category": "Other",
            "sub_category": "miscellaneous",
            "main_confidence": 0.0,
            "sub_confidence": 0.0
        }

# Main function
def process_reviews(preprocessed_csv, batch_size=100):
    try:
        # Load preprocessed reviews
        df = pd.read_csv(preprocessed_csv)
        df = df[['userName', 'content', 'translated_content', 'score', 'date', 'source', 'year']].dropna(subset=['translated_content'])
        logging.info(f"Loaded {len(df)} preprocessed reviews")

        feature_reviews = []
        classified_results = []

        # Process in batches
        for i in range(0, len(df), batch_size):
            batch = df.iloc[i:i+batch_size]
            logging.info(f"Processing batch {i//batch_size + 1}")

            for _, row in batch.iterrows():
                original = row["content"]
                review = row["translated_content"]
                year = row["year"] if not pd.isna(row["year"]) else "Unknown"
                score = row["score"]
                source = row["source"]

                if is_feature_request(review):
                    feature_reviews.append({
                        "original": original,
                        "translated": review,
                        "score": score,
                        "year": year,
                        "source": source
                    })
                    result = classify_feature_request(review)
                    result.update({
                        "original": original,
                        "translated": review,
                        "score": score,
                        "year": year,
                        "source": source
                    })
                    classified_results.append(result)

        feature_df = pd.DataFrame(feature_reviews)
        classified_df = pd.DataFrame(classified_results)

        # Save results
        feature_df.to_csv("feature_requests_only.csv", index=False)
        classified_df.to_csv("categorized_feature_requests.csv", index=False)
        files.download("feature_requests_only.csv")
        files.download("categorized_feature_requests.csv")

        # Generate and save summary
        if not classified_df.empty:
            summary = classified_df.groupby(['main_category', 'sub_category']).size().reset_index(name='count')
            summary.to_csv("feature_requests_summary.csv", index=False)
            files.download("feature_requests_summary.csv")
            logging.info("Summary saved to feature_requests_summary.csv")

        logging.info(f"Processed {len(feature_reviews)} feature requests")
        return feature_df, classified_df

    except Exception as e:
        logging.error(f"Error processing reviews: {e}")
        raise

# Example usage
if __name__ == "__main__":
    try:
        feature_df, classified_df = process_reviews("myslt_preprocessed_reviews.csv")
        print("Processing complete. Files downloaded: feature_requests_only.csv, categorized_feature_requests.csv, feature_requests_summary.csv")
        print("\nSample of categorized feature requests:")
        print(classified_df.head())
    except Exception as e:
        print(f"Failed to process reviews: {e}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Processing complete. Files downloaded: feature_requests_only.csv, categorized_feature_requests.csv, feature_requests_summary.csv

Sample of categorized feature requests:
                                              review main_category  \
0  everything ok, some features can be better lik...         UI/UX   
1  The app is improving, please add dark mode and...         UI/UX   
2                               Add a refresh button         UI/UX   
3            need Wifi router password change option         Login   
4  Atleast make a widget for the app to display t...         UI/UX   

              sub_category  main_confidence  sub_confidence  \
0        simplified layout             0.22            0.40   
1                dark mode             0.59            0.83   
2  navigation improvements             0.69            0.74   
3        password recovery             0.30            0.65   
4      theme customization             0.67            0.37   

                              

Apply Classification to Scraped Reviews (CSV)

# update within 24hrs

In [8]:
#Required installations
# Run these once in your environment (e.g., Colab or local)
!pip install google-play-scraper deep-translator transformers pandas --quiet

# Full Pipeline: Scrape, Translate, Classify, and Save
from google_play_scraper import reviews_all
from transformers import pipeline
from deep_translator import GoogleTranslator
import pandas as pd
from datetime import datetime, timedelta
import os

# Zero-Shot Classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Main categories
main_categories = [
    "User Interface or User Experience issues",
    "Performance problems",
    "Network issues",
    "Billing and Payment problems",
    "Login or Security issues",
    "Other"
]

category_mapping = {
    "User Interface or User Experience issues": "UI/UX",
    "Performance problems": "Performance",
    "Network issues": "Network",
    "Billing and Payment problems": "Billing",
    "Login or Security issues": "Login/Security",
    "Other": "Other"
}

# Subcategories
sub_categories = {
    "User Interface or User Experience issues": ["dark mode", "navigation issues", "layout problems", "text size", "theme customization"],
    "Performance problems": ["slow loading", "app crash", "freezing", "high battery usage"],
    "Network issues": ["no internet", "slow connection", "timeout", "offline mode not working"],
    "Billing and Payment problems": ["double charge", "bill not updating", "payment failure", "incorrect charges"],
    "Login or Security issues": ["cannot login", "OTP not received", "session expired", "fingerprint login"],
    "Other": ["feature request", "language issue", "general feedback"]
}

# Translate if needed
def translate_to_english(text):
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return text

# Classification function
def classify_main_and_sub(review):
    main_result = classifier(review, main_categories)
    verbose_label = main_result['labels'][0]
    main_category = category_mapping.get(verbose_label, "Other")

    sub_list = sub_categories.get(verbose_label, sub_categories["Other"])
    sub_result = classifier(review, sub_list)
    sub_category = sub_result['labels'][0]

    return {
        "review": review,
        "main_category": main_category,
        "sub_category": sub_category,
        "confidence_main": round(main_result['scores'][0], 2),
        "confidence_sub": round(sub_result['scores'][0], 2)
    }

# Fetch Google Play reviews from last 24 hours
def fetch_new_reviews_last_24hrs():
    all_reviews = reviews_all(
        'com.slt.selfcare',
        lang='en',
        country='lk',
        sleep_milliseconds=1000
    )
    now = datetime.utcnow()
    yesterday = now - timedelta(hours=24)

    reviews_24hr = []
    for review in all_reviews:
        posted_date = review.get('at')
        if posted_date and posted_date >= yesterday:
            reviews_24hr.append(review)

    return reviews_24hr

#  Update the CSV file with new reviews
def update_review_classification_24hr():
    reviews = fetch_new_reviews_last_24hrs()
    if not reviews:
        print("No new reviews in last 24 hours.")
        return

    print(f"Found {len(reviews)} new reviews from last 24 hours...")

    # Load existing if exists
    if os.path.exists("myslt_classified_reviews.csv"):
        df_existing = pd.read_csv("myslt_classified_reviews.csv")
        existing_texts = set(df_existing["review"].dropna())
    else:
        df_existing = pd.DataFrame()
        existing_texts = set()

    new_entries = []
    for rev in reviews:
        raw_text = rev.get("content", "")
        if raw_text and raw_text not in existing_texts:
            eng_text = translate_to_english(raw_text)
            result = classify_main_and_sub(eng_text)
            result["original"] = raw_text
            result["translated"] = eng_text
            result["timestamp"] = rev["at"]
            new_entries.append(result)

    if new_entries:
        df_new = pd.DataFrame(new_entries)
        df_final = pd.concat([df_existing, df_new], ignore_index=True)
        df_final.to_csv("myslt_classified_reviews.csv", index=False)
        print(f"Updated myslt_classified_reviews.csv with {len(new_entries)} new classified reviews.")
    else:
        print("No truly new reviews to classify.")

# Run once per day manually or with scheduler
if __name__ == "__main__":
    update_review_classification_24hr()


Device set to use cuda:0


No new reviews in last 24 hours.


# Summarize Feature Requests

In [ ]:
import pandas as pd
import os

# Load classified results
file_path = "myslt_classified_reviews.csv"

if os.path.exists(file_path):
    df = pd.read_csv(file_path)

    # Group by main and subcategory
    summary = df.groupby(["main_category", "sub_category"]).size().reset_index(name="count")

    # Create plain text summary for email
    report_lines = [" Weekly Feature Request Summary:\n"]
    for _, row in summary.iterrows():
        line = f"- {row['main_category']} > {row['sub_category']}: {row['count']} requests"
        report_lines.append(line)

    email_body = "\n".join(report_lines)
    print(email_body)  # Preview
else:
    print(f"Error: The file '{file_path}' was not found. Please ensure the previous steps to create this file have been executed successfully.")

Error: The file 'myslt_classified_reviews.csv' was not found. Please ensure the previous steps to create this file have been executed successfully.


# Send Email

In [14]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
import os

def send_email(subject, body, to_email, from_email, smtp_server, smtp_port, login, password, attachment_paths=None):
    msg = MIMEMultipart()
    msg["Subject"] = subject
    msg["From"] = from_email
    msg["To"] = ", ".join(to_email)

    msg.attach(MIMEText(body, 'plain'))

    if attachment_paths:
        for attachment_path in attachment_paths:
            if os.path.exists(attachment_path):
                with open(attachment_path, "rb") as attachment:
                    part = MIMEApplication(attachment.read(), _subtype="octet-stream")
                    part.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(attachment_path)}"')
                    msg.attach(part)
            else:
                print(f"Warning: Attachment file not found at {attachment_path}")


    try:
        with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
            server.login(login, password)
            server.sendmail(from_email, to_email, msg.as_string())
        print("Email sent successfully!")
    except smtplib.SMTPAuthenticationError:
        print("SMTP Authentication Error: Make sure your username and password are correct.")
        print("If using Gmail, ensure Less Secure App access is enabled or use an App Password if 2FA is on.")
    except Exception as e:
        print(f"An error occurred while sending email: {e}")




sender_email =  "kaveeshanirmani25@gmail.com"

app_password = "cidkljqictlwmady"

# List of files to attach
attachment_files = ["feature_requests_summary.csv","categorized_feature_requests.csv"]


send_email(
    subject=" Weekly MYSLT Feature Request Report",
    body=email_body,
    to_email=["yehanmanodya12@gmail.com", "kaveeshasharadi@gmail.com"],
    from_email=sender_email,
    smtp_server="smtp.gmail.com",
    smtp_port=465,
    login=sender_email,
    password=app_password,
    attachment_paths=attachment_files # Pass the list of attachment paths
)

Email sent successfully!
